# Guia para a aula experimental

Nesse guia trataremos passo a passo a conexão de um dispositivo com a plataforma Konker.
Para seguir esse roteiro, você precisará de uma conta na Konker, que pode ser criada em https://demo.konkerlabs.net/ e, caso deseje trabalhar com o kit disponibilizado para vocês, será necessário baixar a Arduino IDE e adicionar o suporte a placa de desenvolvimento **NodeMCU**, além de instalar as bibliotecas **PubSubClient** e **ArduinoJSON**. Um guia de instalação da placa pode ser visto na referência https://www.filipeflop.com/blog/programar-nodemcu-com-ide-arduino/ enquanto a instalação das bibliotecas pode ser feita sem dificuldades buscando pelo nome das mesmas na caixa aberta ao selecionar o menu **Sketch** -> **Incluir Biblioteca** -> **Gerenciar Bibliotecas**.

Caso você não tenha participado da aula e, em casa, prefira trabalhar com dispositivos virtuais, não será necessária a instalação da Arduino IDE e todo o procedimento pode ser feito diretamente nesse notebook jupyter. 

**Independente da sua escolha, por favor siga o procedimento do notebook e lembre-se de salvar suas alterações (o atalho Ctrl+s funciona), pois sua avaliação será feita com base nele.**

Vamos iniciar nosso notebook chamando todas as bibliotecas que usaremos. Nessa etapa, nada precisa ser modificado.

In [ ]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import pprint
import numpy as np
import arrow
import requests                                                                                                                                                                                 
import json                                                                                                                                                                                     
from threading import Timer

O próximo passo é definir os endereços que serão usados para consultar e enviar os dados. Nessa etapa ainda não é necessário fazer modificações.

In [ ]:
#Url de publicacao dos dados
pub_url = 'https://data.demo.konkerlabs.net/pub/'
#Url da API
base_api = 'https://api.demo.konkerlabs.net'
#Application padrão
application = 'default'

Agora vamos colocar em duas variáveis o seu usuário e senha da plataforma:

In [ ]:
username = ''
password = ''

O próximo passo é criar 3 dispositivos dentro de sua conta na Konker. Um deles será o dispositivo **LED**, outro será o **Termometro** e o último eu chamarei de **Machine_Learning**, que será um dispositivo virtual que tomará decisões baseadas em Aprendizado de Máquina. Você pode criar os dispositivos com o nome de sua preferência, lembre-se apenas de modificar nas funções abaixo para o nome adequado.
Quando você criar o dispositivo, clique em **Connect** -> **Generate Password** e coloque as credenciais de ambos os dispositivos nas variáveis abaixo:

In [ ]:
led_name = "led"
led_username = ""
led_password = ""

termometro_name = "termometro"
termometro_username = ""
termometro_password = ""

ML_name = "Machine_Learning"
ML_username = ""
ML_password = ""

Nesse momento, precisaremos fazer uma divisão no procedimento: caso você esteja fazendo essa aula com o kit de hardware, siga os procedimentos sob o título **Usando o Kit de Hardware** e caso você esteja fora da aula fazendo o procedimento simulado, siga o procedimento **Usando dispositivos simulados**.

## Usando o Kit de Hardware
Esse é o momento de você baixar o código dos dispositivos disponível em nosso GitHub: https://github.com/KonkerLabs/arduino_examples. Você deve baixar o código do **LED_Device_MQTT** e **Termometro_MQTT**. Depois de baixar o código, abra primeiramente o código do Termometro na Arduino IDE e mude os parâmetros de rede Wifi, canal de publicação (minha sugestão é usar *temperatura*) credenciais do dispositivo. Dica: você acabou de escrever essas credenciais nas variáveis acima como "cola" para esse momento, pois elas não serão usadas no notebook :-).

Agora é hora de montar o seu termômetro!
Basta seguir a ilustração abaixo:
![term](https://raw.githubusercontent.com/KonkerLabs/arduino_examples/master/Termometro_MQTT/term.jpg "Termômetro")

Com o dispositivo montado, o próximo passo é compilar e gravar o Firmware. Lembre-se de mudar a board na Arduino IDE para **NodeMCU v1.0**.

Caso tudo tenha dado certo até o momento, você começará a observar os dados sendo enviados para a plataforma. Entre na Guia de Devices e procure Messages do seu dispositivo termômetro. Você deve ver os dados de temperatura chegando então vamos fazer o primeiro experimento do dia.

**Escreva abaixo a temperatura que seu termômetro está medindo no ambiente (observada na plataforma):**

**Segure o termistor com seus dedos até a temperatura estabilizar. Escreva abaixo a temperatura que está sendo medida na sua mão:**

Agora vamos trabalhar com rotas entre dois dispositivos. Para tornar essa atividade mais interativa, escolha uma dupla na sala de aula. Um de vocês precisará montar agora o dispositivo LED. Vocês receberam 2 protoboards para ter a opção de montar os dois dispositivos ao mesmo tempo: basta colocar cada protoboard em um dos lados do NodeMCU.

O dispositivo LED deve ser montado seguindo o diagrama abaixo:

![term](https://raw.githubusercontent.com/KonkerLabs/arduino_examples/master/LED_Device_MQTT/leds.jpg "Termômetro")

O próximo passo é conectar o dispositivo na plataforma usando as credenciais do dispositivo LED (use a cola acima), o canal que ele irá subscrever (minha sugestão é usar *led*) e criar uma rota entre os dispositivos termômetro e LED. Para isso entre na aba **Event Routes** e clique em **New** do lado direito superior.

A rota pode ser criada com o número de sua preferência, lembrando apenas de colocar o **Income Device** como o seu dispositivo termômetro e o seu **Outgoing Device** como seu dispositivo LED. Caso você tenha usado o canal de envio **temperatura** esse será o seu **Income Channel** da mesma forma que caso seu dispositivo LED esteja subscrevendo no canal **led**, esse será o seu **Outgoing Channel**.

Caso tudo tenha funcionado, você deve estar vendo as mensagens do termômetro chegarem no dispositivo LED.

**Escreva abaixo qual o resultado observado lendo a porta serial do dispositivo LED**

----> Escreva nessa célula <----

**Escreva abaixo quantos LEDs estão acesos na temperatura ambiente**

----> Escreva nessa célula <----

**Qual o threshold você precisaria colocar para acender um LED na temperatura ambiente e 2 LEDs com sua mão no termistor?**

----> Escreva nessa célula <----

Bem interessante, mas como você pode ver, uma abordagem assim tem um problema fundamental: ela só funciona quando você conhece quais as temperaturas que você está discriminando, nesse caso ambiente versus temperatura do corpo humano. Caso o problema fosse sobre coisas desconhecidas, como discriminar entre a temperatura normal de trabalho de uma turbina CFM56-3B-1 e dela superaquecida, essa técnica não funcionaria. Para isso temos técnicas de separação que podem aprender de forma não supervisionada com o seu dataset, como a técnica de KMeans que mostraremos no decorrer do notebook.

## Usando dispositivos simulados
Caso você tenha feito a parte acima com o dispositivo real, pode ignorar essa seção e ir diretamente para **Usando a API**.

Vamos iniciar a parte dos dispositivos virtuais importando as bibliotecas que iremos utilizar.

In [ ]:
import paho.mqtt.client as mqtt
import random
import time

Da mesma forma que fizemos com os dispositivos reais, vamos utilizar MQTT para o enviar os dados de nosso dispositivo virtual para a plataforma. Vamos iniciar o trabalho simulando um termômetro enviando dados por 75 segundos, nos quais 50 são com temperatura ambiente e 25 com uma temperatura compatível com o corpo humano. Abaixo estão as funções que usaremos para isso.

In [ ]:
def on_connect(mqttc, a, userdata, rc):
    global termometro_username
    print("Connected with result code "+str(rc))
    mqttc.subscribe("sub/" + led_username + "/led")
       
def on_message(mqttc, userdata, msg):
    json_data = msg.payload.decode('utf-8')
    temperature = json.loads(json_data)['value']
    print(float(temperature))
    if (float(temperature)>30.0):
        print('Mensagem recebida! Estado dos LEDS virtuais = [1,1,1]')
    elif (float(temperature)>20.0):
        print('Mensagem recebida! Estado dos LEDS virtuais = [1,1,0]')
    elif (float(temperature)>10.0):
        print('Mensagem recebida! Estado dos LEDS virtuais = [1,0,0]')
    else:
        print('Mensagem recebida! Estado dos LEDS virtuais = [0,0,0]')
    
def publicar_temperatura(mqttc_term):
    for i in range(0,25):
        mqttc_term.publish("pub/" + termometro_username + "/temperatura", json.dumps({"deviceId":"My_favorite_thermometer", "metric": "Celsius", "value": float("{0:.2f}".format(random.gauss(25,0.5)))}))
        time.sleep(1)
    for i in range(0,25):
        mqttc_term.publish("pub/" + termometro_username + "/temperatura", json.dumps({"deviceId":"My_favorite_thermometer", "metric": "Celsius", "value": float("{0:.2f}".format(random.gauss(32,0.5)))}))
        time.sleep(1)
    for i in range(0,25):
        mqttc_term.publish("pub/" + termometro_username + "/temperatura", json.dumps({"deviceId":"My_favorite_thermometer", "metric": "Celsius", "value": float("{0:.2f}".format(random.gauss(25,0.5)))}))
        time.sleep(1)


Vamos agora conectar nosso dispositivo virtual na plataforma:

In [ ]:
mqttc_term = mqtt.Client(termometro_username)
mqttc_term.username_pw_set(termometro_username, termometro_password)
mqttc_term.connect("mqtt.demo.konkerlabs.net", 1883)
mqttc_term.loop_start()

Por ultimo vamos ativar o device virtual. Se tudo estiver correto, ao rodar a próxima célula, você começará a receber dados na plataforma.

In [ ]:
publicar_temperatura(mqttc_term)

**Escreva abaixo a temperatura que seu termômetro virtual está medindo como "ambiente" (observando na plataforma):**

----> Escreva nessa célula <----

**Escreva abaixo a temperatura que está sendo medida como "corpo humano":**

----> Escreva nessa célula <----

Agora vamos trabalhar com rotas entre dois dispositivos virtuais. Vamos criar um dispositivo virtual que fará subscrição na plataforma como dispositivo LED e imprimirá as mensagens recebidas por ele na tela

In [ ]:
mqttc_led = mqtt.Client(led_username)
mqttc_led.username_pw_set(led_username, led_password)
mqttc_led.on_message = on_message
mqttc_led.on_connect = on_connect
mqttc_led.connect("mqtt.demo.konkerlabs.net", 1883)
mqttc_led.loop_start()

O próximo passo é criar uma rota entre os dispositivos termômetro e LED. Para isso entre na aba **Event Routes** e clique em **New** do lado direito superior.

A rota pode ser criada com o nome de sua preferência, lembrando apenas de colocar o **Income Device** como o seu dispositivo termômetro e o seu **Outgoing Device** como seu dispositivo LED. Caso você tenha usado o canal de envio **temperatura** esse será o seu **Income Channel** da mesma forma que caso seu dispositivo LED esteja subscrevendo no canal **led**, esse será o seu **Outgoing Channel**. Esses são os canais que definimos nas funções acima, de forma que caso você não tenha modificado as funções, esses serão os canais que devem ser utilizados na plataforma.

Caso tudo tenha funcionado corretamente, uma vez que rodemos o termômetro virtual, os dados relativos ao estado dos LEDs virtuais devem ser impressos na tela. Vamos testar?

In [ ]:
publicar_temperatura(mqttc_term)

**Escreva abaixo quantos LEDs virtuais estão acesos (com valor 1) na temperatura ambiente virtual**

----> Escreva nessa célula <----

**Escreva abaixo quantos LEDs virtuais estão acesos (com valor 1) na temperatura do corpo humano virtual**

----> Escreva nessa célula <---- 

Agora vamos passar para o uso da API e da aplicação da técnica de Machine Learning nos dados.

## Usando a API
Para iniciar esse trabalho, vamos primeiro conectar na API da Konker. A API usa OAuth2, então primeiro vamos obter as credenciais.

In [ ]:
client = BackendApplicationClient(client_id=username)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='{}/v1/oauth/token'.format(base_api),
                                       client_id=username,
                                       client_secret=password)

Ótimo! Agora nos podemos começar a usar a API. Caso você queira mais detalhes de sua utilização, pode encontrar documentação em: https://api.demo.konkerlabs.net .

Vamos começar listando os dispositivos registrados no seu usuário.

In [ ]:
devices = oauth.get("https://api.demo.konkerlabs.net/v1/{}/devices/".format(application)).json()['result']
for dev in devices:
    print(dev)

Vamos procurar pelo dispositivo Termometro e Machine_Learning na sua lista de dispositivos:

In [ ]:
guid_ml=""
guid_term=""
for dev in devices:
    if dev['name'] == termometro_name:
        guid_term = dev['guid']
    elif dev['name'] == ML_name:
        guid_ml = dev['guid']

print("O GUID do dispositivo Termômetro é: "+ guid_term)
print("O GUID do dispositivo Machine Learning é: "+ guid_ml)

Caso você consiga ver o GUID dos dois dispositivos, significa que está tudo funcionando bem. Caso algum dos GUIDs não apareça, revise o nome do dispositivo no Notebook e o nome escolhido na plataforma para garantir que eles possuem a mesma grafia.

Agora vamos usar a API para pegar os dados enviados pelo dispositivo termômetro hoje. Caso você tenha escolhido outro canal para envio dos dados, por favor modifique a variável canal na próxima célula.

In [ ]:
canal = 'temperatura'
dt_start = arrow.utcnow().to('America/Sao_Paulo').floor('day')
dt_start = dt_start.shift(days=-10)
stats = oauth.get("https://api.demo.konkerlabs.net/v1/{}/incomingEvents?q=device:{} channel:{} timestamp:>{}&sort=oldest&limit=10000".format(application,guid_term,canal,dt_start.isoformat())).json()['result']
print(stats)

Caso tudo tenha funcionado como esperado, você deve estar vendo seus dados de temperatura logo acima. Para facilitar a visualização e análise dos dados, podemos transformar em um formato tabular com o Pandas.

In [ ]:
from pandas.io.json import json_normalize
stats_df = json_normalize(stats).set_index('timestamp')
stats_df = stats_df[3:]
stats_df

Ótimo! Agora os dados estão em um formato mais fácil de ler. Mas podemos também fazer um gráfico bem simples!

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(15,4))

stats_df['payload.value'].plot();

**O gráfico acima ilustra corretamente seu experimento? Descreva abaixo as características esperadas e se de fato elas foram observadas no gráfico**

----> Escreva nessa célula <----

Agora começa a parte final desse trabalho. Vamos rodar um algoritmo conhecido com KMeans de aprendizado não supervisionado tentando encontrar os dois clusters que melhor separam nosso dataset. Como você pode observar abaixo, estamos usando a biblioteca SKLearn do Python para isso.

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
kmeans.fit(stats_df['payload.value'].values.reshape(-1, 1))

Nós colocamos como condição a separação em 2 clusters. Vamos ver qual a previsão feita sobre os dados adquiridos:

In [ ]:
predictions = kmeans.predict(stats_df['payload.value'].values.reshape(-1, 1))
predictions

Vamos observar a temperatura média do primeiro cluster de dados (grupo 0):

In [ ]:
print('Temperatura média do grupo 0: ' + str(stats_df.loc[predictions == 0]['payload.value'].mean()) + ' C')

E agora a temperatura média do segundo cluster (grupo 1):

In [ ]:
print('Temperatura média do grupo 1: ' + str(stats_df.loc[predictions == 1]['payload.value'].mean()) + ' C')

**Escreva abaixo qual dos dois grupos parece representar melhor a temperatura ambiente**

----> Escreva nessa célula <---- 

Vamos ver agora como se comportam os dois clusters encontrados em um gráfico. 

Nota: Nessa segunda versão do Notebook estou usando a biblioteca Bokeh para tentar evitar o gargalo em processamento gerado pelo Matplotlib na sala de aula. Caso você ainda encontre problemas em gerar o gráfico, por favor me envie um email em luis@konkerlabs.com

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import pandas as pd
output_notebook()

In [ ]:
p = figure(plot_width=820, plot_height=400, x_axis_type="datetime",
           title="Clusters de temperatura encontrados pelo método KMeans",
           x_axis_label='Tempo', 
           y_axis_label='Temperatura [Celsius]')

p.title.text_font_size = '18pt'
p.xaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label_text_font_size = "14pt"

x = np.array(pd.to_datetime(stats_df.index))
y = np.array(stats_df['payload.value'])
n_y = np.multiply(np.array(stats_df['payload.value']),predictions)
n_y = np.clip(n_y,np.min(y),np.max(y))

# add a line renderer
p.line(x, y, line_width=2)
p.patch(x,n_y,color="red",alpha = 0.5,line_width=0)

show(p) # show the results

Observe que em momento algum você precisou escolher um threshold para a divisão entre os clusters de temperatura. Muito embora esses dados representem coisas muito bem conhecidas, elas poderiam representar a situação normal de trabalho de uma máquina e a situação de superaquecimento.

Agora vamos usar o modelo gerado pelo KMeans para criar um dispositivo virtual que classifique cada dado como "temperatura humana" e "temperatura ambiente". Tipicamente em um sistema de IoT em produção, essa função não seria feita por um dispositivo virtual e sim por uma aplicação rodando na nuvem. A ideia aqui é apenas ilustrar esse processo usando as ferramentas mais simples da plataforma.

Para iniciar o processo, vamos criar algumas funções para nos ajudar:

In [ ]:
import paho.mqtt.client as mqtt
import random
import time

def on_connect(mqttc, a, userdata, rc):
    global termometro_username
    print("Connected with result code "+str(rc))
    mqttc.subscribe("sub/" + led_username + "/led")
       
def on_message(mqttc, userdata, msg):
    json_data = msg.payload.decode('utf-8')
    temperature = json.loads(json_data)['value']
    print(float(temperature))
    if (float(temperature)>30.0):
        print('Mensagem recebida! Estado dos LEDS virtuais = [1,1,1]')
    elif (float(temperature)>20.0):
        print('Mensagem recebida! Estado dos LEDS virtuais = [1,1,0]')
    elif (float(temperature)>10.0):
        print('Mensagem recebida! Estado dos LEDS virtuais = [1,0,0]')
    else:
        print('Mensagem recebida! Estado dos LEDS virtuais = [0,0,0]')
        
def on_connect_ml(mqttc, a, userdata, rc):
    global ML_username
    print("Connected with result code "+str(rc))
    mqttc.subscribe("sub/" + ML_username + "/in")
       
def on_message_ml(mqttc, userdata, msg):
    global kmeans
    global ML_username
    json_data = msg.payload.decode('utf-8')
    temperature = json.loads(json_data)['value']
    pred = kmeans.predict(temperature)[0]
    if (pred==0):
        send = 15.0
    elif (pred==1):
        send = 100.0
    mqttc.publish("pub/" + ML_username + "/out", json.dumps({"deviceId":"My_favorite_ML_Model", "metric": "Model_Output", "value": send}))

def publicar_temperatura(mqttc_term):
    for i in range(0,25):
        mqttc_term.publish("pub/" + termometro_username + "/temperatura", json.dumps({"deviceId":"My_favorite_thermometer", "metric": "Celsius", "value": float("{0:.2f}".format(random.gauss(25,0.5)))}))
        time.sleep(1)
    for i in range(0,25):
        mqttc_term.publish("pub/" + termometro_username + "/temperatura", json.dumps({"deviceId":"My_favorite_thermometer", "metric": "Celsius", "value": float("{0:.2f}".format(random.gauss(32,0.5)))}))
        time.sleep(1)
    for i in range(0,25):
        mqttc_term.publish("pub/" + termometro_username + "/temperatura", json.dumps({"deviceId":"My_favorite_thermometer", "metric": "Celsius", "value": float("{0:.2f}".format(random.gauss(25,0.5)))}))
        time.sleep(1)

A função **on_message_ml** recebe as mensagens contendo a temperatura e usa esse valor como entrada no modelo gerado pelo KMeans. Caso a saída do modelo seja no cluster 0, será enviada uma mensagem para o canal **out** contendo o valor 15.0. Caso a saída seja cluster 1, será enviada uma mensagem contendo o valor 100.0. Esses valores foram arbitrariamente escolhidos para manter compatibilidade com o Software do Dispositivo LED, que no primeiro valor acenderá um LED e no segundo valor acenderá os 3 LEDs.

Agora precisamos criar as rotas entre os dispositivos na plataforma. Vamos começar editando a rota que havíamos criado: nela, modifique o **Outgoing Device** para o dispositivo **Machine_Learning** e o **Outgoing Channel** para **in**.

Agora você deve criar uma nova rota com nome de sua preferência, colocando o **Income Device** como o seu dispositivo **Machine_Learning** e o seu **Outgoing Device** como seu dispositivo LED. Coloque **out** como o seu **Income Channel** e, caso seu dispositivo LED esteja subscrevendo no canal **led**, esse será o seu **Outgoing Channel**. Esses são os canais que definimos no início do arquivo.

Se tudo estiver correto, podemos agora ativar nosso dispositivo virtual e observar as mensagens trafegando pela plataforma.

Nota: Nessa segunda versão do Notebook, são criados os 3 dispositivos virtuais (LED, Termômetro e ML) de forma que vocês possam testar em casa o resultado. Vocês também podem inserir algum dispositivo real nesse teste: por exemplo se quiserem usar hardware termistor, basta comentar as linhas iniciadas em **mqttc_term** antes de rodar o código abaixo e enviar os dados reais do termistor para a plataforma. Você receberá um feedback do estado dos seus LED "virtuais" no notebook. Por outro lado, se quiser usar o LED de hardware, basta comentar as linhas iniciadas em **mqttc_led** e manter o seu dispositivo de hardware conectado na plataforma.

**Nota Importante**: Caso você receba mais do que duas mensagens: **Connected with result code 0**, significa que ocorreu algo errado com sua conexão MQTT e ela está sofrendo reconexões. Caso isso ocorra, por favor reinicie o Kernel do seu notebook, clicando em **Restart** no menu **Kernel**.

In [ ]:
mqttc_ml = mqtt.Client(ML_username)
mqttc_ml.username_pw_set(ML_username, ML_password)
mqttc_ml.on_message = on_message_ml
mqttc_ml.on_connect = on_connect_ml
mqttc_ml.loop_stop(force=True)
mqttc_ml.disconnect()
time.sleep(2)
mqttc_ml.connect("mqtt.demo.konkerlabs.net", 1883)
mqttc_ml.loop_start()

mqttc_term = mqtt.Client(termometro_username)
mqttc_term.username_pw_set(termometro_username, termometro_password)
mqttc_term.loop_stop(force=True)
mqttc_term.disconnect()
time.sleep(2)
mqttc_term.connect("mqtt.demo.konkerlabs.net", 1883)
mqttc_term.loop_start()

mqttc_led = mqtt.Client(led_username)
mqttc_led.username_pw_set(led_username, led_password)
mqttc_led.on_message = on_message
mqttc_led.on_connect = on_connect
mqttc_led.loop_stop(force=True)
mqttc_led.disconnect()
time.sleep(2)
mqttc_led.connect("mqtt.demo.konkerlabs.net", 1883)
mqttc_led.loop_start()

**Caso você esteja usando o Kit de Hardware: Qual comportamento você observa no dispositivo LED quando segura o Termistor?**

----> Escreva nessa célula <----

Caso você esteja usando o termômetro virtual, precisaremos ativa-lo mais uma vez:

In [ ]:
publicar_temperatura(mqttc_term)

**Se você está usando o dispositivo virtual: Qual o comportamento observado na resposta dos LEDs simulados?**

----> Escreva nessa célula <---- 

### Esse é o fim dessa atividade. Espero que tenham gostado e que esses procedimentos simples possam dispertar novas ideias para o uso de IoT em problemas reais.